-- QUESTION 2 --

a) An example string of 100 nucleotides is created and k-mers of values 3,4,5,20,30 and 50 are found.

In [ ]:
original_string = "GTAGATCGTACGTACTAGCGATGCTGAGGACTGATCGACGTAGCTAGCGGATCGATCGAGGACTTCTTTAGCTAGCTAACATGTCGATGCTAGCTAGCTA"
print(len(original_string))

In [ ]:
def kmer_calculator(seq, k):
    kmers = []
    for i in range(len(seq) - k + 1):
        kmer = seq[i:i+k]
        kmers.append(kmer)
    return kmers

In [ ]:
print(f"K-mers for k=3: {kmer_calculator(original_string, 3)}")
print(f"K-mers for k=4: {kmer_calculator(original_string, 4)}")
print(f"K-mers for k=5: {kmer_calculator(original_string, 5)}")
print(f"K-mers for k=20: {kmer_calculator(original_string, 20)}")
print(f"K-mers for k=30: {kmer_calculator(original_string, 30)}")
print(f"K-mers for k=50: {kmer_calculator(original_string, 50)}")

b) Original string is reconstructed using k-mers generated.

In [ ]:
k3 = kmer_calculator(original_string, 3)
k4 = kmer_calculator(original_string, 4)
k5 = kmer_calculator(original_string, 5)
k20 = kmer_calculator(original_string, 20)
k30 = kmer_calculator(original_string, 30)
k50 = kmer_calculator(original_string, 50)


#Random is imported for shuffling kmers.
import random
import numpy as np

def reconstruct_string_from_kmers(kmers):
    # Function to build the Eulerian De Bruijn graph from k-mers and find the start node.
    def build_eulerian_de_bruijn_graph(kmers):
        graph = {}
        indegree = {}
        outdegree = {}


         # Iterate through each k-mer to build the graph.
        for kmer in kmers:
            prefix = kmer[:-1]
            suffix = kmer[1:]

            # Initialize vertex information if not present.
            if prefix not in graph:
                graph[prefix] = []
            if prefix not in outdegree:
                outdegree[prefix] = 0
            if suffix not in indegree:
                indegree[suffix] = 0

            # Connect prefix to suffix in the graph.
            graph[prefix].append(suffix)
            outdegree[prefix] += 1
            indegree[suffix] += 1
    
        # Find potential start and end nodes by comparing in-degrees and out-degrees.
        start, end = None, None
        for node in set(indegree.keys()).union(outdegree.keys()):
            indeg = indegree.get(node, 0)
            outdeg = outdegree.get(node, 0)
    
            if indeg < outdeg:  # More outgoing edges means potential start node.
                start = node
            elif indeg > outdeg:  # More incoming edges means potential end node.
                end = node
    
        # Addition of an edge from end node to start node to make it Eulerian path.
        if start and end:
            if end not in graph:
                graph[end] = []
            graph[end].append(start)
    
        return graph, start
    
    def eulerian_cycle(graph, start_vertex):
        # Helper function to find a vertex in the cycle with unexplored edges.
        def find_new_start(cycle):
            for vertex, _ in cycle:
                if graph[vertex]:  # If there are unexplored edges.
                    return vertex
            return None
    
        # Adjust the starting point of the cycle.
        if start_vertex is None or start_vertex not in graph or not graph[start_vertex]:
            return []  # No valid start or no edges in the graph.
    
        cycle = [(start_vertex, graph[start_vertex].pop())]
        current_vertex = cycle[0][1]
    
        # Continue walking randomly until the start vertex is reached as algorithm suggests.
        while current_vertex != start_vertex:
            next_vertex = graph[current_vertex].pop()
            cycle.append((current_vertex, next_vertex))
            current_vertex = next_vertex
    
        # Continue forming the cycle until all edges are visited.
        while any(graph.values()):
            new_start = find_new_start(cycle)
            if new_start is None:
                break  # No unexplored edges left in the cycle.
    
            # Start a new cycle from the new start vertex.
            current_vertex = new_start
            new_cycle = [(current_vertex, graph[current_vertex].pop())]
            current_vertex = new_cycle[0][1]
    
            # Complete the new cycle.
            while current_vertex != new_start:
                next_vertex = graph[current_vertex].pop()
                new_cycle.append((current_vertex, next_vertex))
                current_vertex = next_vertex
    
            # Find the index to merge the new cycle into the original cycle.
            merge_index = next(i for i, edge in enumerate(cycle) if edge[0] == new_start)
            cycle = cycle[:merge_index] + new_cycle + cycle[merge_index:]
    
        return cycle
    
    
    #To be sure that the algorithm doesn't just add the serial kmers that are already in order, we shuffle the kmers.
    random.shuffle(kmers)
    
    # Generate the Eulerian De Bruijn graph and find the start node. 
    de_bruijn_graph, start_node = build_eulerian_de_bruijn_graph(kmers)
        
    # Find the Eulerian cycle starting from the start node.
    eulerian_cycle = eulerian_cycle(de_bruijn_graph, start_node)
        
    #The edge between the last node of eulerian cycle and start node is removed to create the eulerian path.
    eulerian_path = eulerian_cycle[:-1] 
    reconstructed_string = ''
    reconstructed_string += eulerian_path[0][0]
    for _, suffix in eulerian_path:
        reconstructed_string += suffix[-1]

    return reconstructed_string
    

print(reconstruct_string_from_kmers(k3))
print(reconstruct_string_from_kmers(k4))
print(reconstruct_string_from_kmers(k5))
print(reconstruct_string_from_kmers(k20))
print(reconstruct_string_from_kmers(k30))
print(reconstruct_string_from_kmers(k50))

# Calculate percent identity with the original string.
# Change the k-mer length by changing k3,k4,k5....
matching_positions = sum(original_string[i] == reconstruct_string_from_kmers(k3)[i] for i in range(len(original_string)))
percent_identity = (matching_positions / len(original_string)) * 100
print(f"Percent Identity for k=3: {percent_identity:.2f}%")


c) I used De Bruijn Graph with Eulerian path reconstruction algorithm to reconstruct DNA sequences using k-mers. The reconstruction process involves shuffling the k-mers to prevent bias, constructing an Eulerian De Bruijn graph from the shuffled k-mers, and then finding an Eulerian cycle in the graph. We are trying to create a balanced graph which avoids already visited edges by randomly walking. The last edge in the Eulerian cycle is removed to create an Eulerian path, representing the reconstructed DNA sequence. The code prints the reconstructed sequences for different k-mer lengths (3,4,5,20,30,50) and provides insights into the efficiency of the reconstruction algorithm for various fragment sizes. 

d) The reconstructed string doesn't always match the original one as can be concluded from the percent identities with the original string. As the kmer length increases, the similarity to original string increases. It is expected because long kmers give more information about the sequence while handling short kmers is hard since the possibilities also increase.